In [1]:
cd ..

/home/jovyan/dsi_plus/Projects/Project-4-semantic_search


In [2]:
%run lib/__init__.py
%matplotlib inline

In [ ]:
!pip install psycopg2

In [ ]:
!pip install nltk

In [3]:
myWiki=MyWikiDB()

In [4]:
pages_cllr=myWiki.dbr.page_collection
pages_cllr.find().count()

1106

In [5]:
cursor=pages_cllr.find()
pages=list(cursor)
corpus_df=pd.DataFrame(pages[:1000])
corpus_df.head()

,_id,category_list,extract,ns,pageid,pull_category,title
0,43385931,[Category:All articles lacking in-text citatio...,data exploration approach similar initial datu...,0,43385931,Category:Machine learning,Data exploration
1,49082762,"[Category:Artificial intelligence, Category:CS...",these dataset use machine learn research cite ...,0,49082762,Category:Machine learning,List of datasets for machine learning research
2,233488,[Category:All articles with unsourced statemen...,machine learn field computer science give comp...,0,233488,Category:Machine learning,Machine learning
3,53587467,"[Category:Artificial intelligence, Category:Co...",the follow outline provide overview topical gu...,0,53587467,Category:Machine learning,Outline of machine learning
4,3771060,"[Category:Data mining, Category:Machine learni...",the accuracy paradox predictive analytic state...,0,3771060,Category:Machine learning,Accuracy paradox


In [6]:
def pull_wiki_page(page):
    '''
    pull page extract and category list for a specified page using 
    wikipedia API
    '''
    
    #pull page extract
    params_p = {"action": "query",
            "titles": page,
            "prop": 'extracts' ,
            "format": 'json' ,
        }
            
    response_p = requests.get("https://en.wikipedia.org/w/api.php", params = params_p)
    data = response_p.json()
    
    #pull page categories
    params_c = {"action": "query",
            "titles": page,
            "prop": 'categories' ,
            "format": 'json' ,
            }
            
    response_c = requests.get("https://en.wikipedia.org/w/api.php", params = params_c)
    cats = response_c.json()
            
    return data , cats

In [7]:
import re 
import spacy

In [10]:
import nltk
from nltk.corpus import stopwords
nlp=spacy.load('en')
nltk.download('stopwords')
nltk_stop=stopwords.words('english')
nltk_stop.append('displaystyle')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
nltk_stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 '

In [ ]:
def cleaner(text):
    ''' Clean text data, apply spacy lemmatization and nltk stop words'''
    text = re.sub('<* />','',text)
    text = re.sub('<.*>.*</.*>','', text)
    text = re.sub('[\d]',' ',text)
    text = re.sub('{*}',' ',text)
    text = re.sub('[\n]',' ',text)
    text = re.sub('[^a-zA-Z ]',' ',text)
    text = ' '.join(i.lemma_ for i in nlp(text)
                    if i.orth_ not in nltk_stop)
    text = ' '.join(text.split())
    return text

In [ ]:
def read_articles( pageid, page):
        '''
        read page extract and category list from wikipedi 
        API query and add to page dict
        '''
        data, cats=pull_wiki_page(page)
        
        # first data clean - extra from json query and parse html
        article=data['query']['pages'][pageid]['extract']
        soup = BeautifulSoup(article, 'html.parser')
        extract=soup.get_text()
        
        # 2nd data clean including lemmatization and stop words
        
        extract_clean=cleaner(extract)

        #cats_p=[c['title'] for c in cats['query']['pages'][str(page['pageid'])]['categories']]

        #page['category_list']=cats_p
        #page['extract']=extract_clean
        
        #Add _id field as pageid - will be used as a unique index by Mongo
        #page['_id']=page['pageid']
        
        return extract_clean 

In [ ]:
read_articles('3771060', 'Accuracy paradox')